https://www.blog.pythonlibrary.org/2016/04/20/python-201-an-intro-to-itertools/`

In [4]:
from itertools import *
import numpy as np

In [5]:


print('Stop at 5:')
for i in islice(count(), 5):
    print(i)

print('Start at 5, Stop at 10:')
for i in islice(count(), 5, 10):
    print(i)

print('By tens to 100:')
for i in islice(count(), 0, 100, 10):
    print(i)

Stop at 5:
0
1
2
3
4
Start at 5, Stop at 10:
5
6
7
8
9
By tens to 100:
0
10
20
30
40
50
60
70
80
90


**Simulating the feature batch generator**

In [6]:
def get_feature_generator():
    for inputs in count():
        yield [inputs, inputs+10], [inputs+100] 

In [7]:
batch_size = 8

In [8]:
feature_generator = get_feature_generator()
batched_features = list(islice(feature_generator, batch_size))

In [9]:
batched_features

[([0, 10], [100]),
 ([1, 11], [101]),
 ([2, 12], [102]),
 ([3, 13], [103]),
 ([4, 14], [104]),
 ([5, 15], [105]),
 ([6, 16], [106]),
 ([7, 17], [107])]

In [36]:
flattened_inputs= [[0, 10], [1, 11], [2, 12], [3, 13], [4, 14], [5, 15], [6, 16], [7, 17]]
list(zip(*flattened_inputs))

[(0, 1, 2, 3, 4, 5, 6, 7), (10, 11, 12, 13, 14, 15, 16, 17)]

In [26]:
def get_batch_generator(get_feature_generator, batch_size):
    """
    Convenience function that, when called, produces a generator that yields
    individual features as numpy arrays into a generator
    that yields batches of features.
    :param get_feature_generator:  numpy array generator
        The feature_generator should be an infinite generator that outputs
        individual training features (as numpy arrays in this codebase,
        but any iterable works). The expected format is:
        ((input0, input1,...), (target0, target1, ...))
    :param batch_size: : int, optional
        The size of each batch. Depending on how many
        features there are in the dataset, the last batch
        may have less features.
    :return: returns a tuple of 2 tuples
        The expected return schema is:
        ((input0, input1, ...), (target0, target1, ...),
        where each of "input*" and "target*" are numpy arrays.
        The number of rows in each input and target numpy array
        should be the same as the batch size.
    """

    # batched_features is a list of batch_size features, where each
    # feature is a tuple ((inputs), targets)
    feature_generator = get_feature_generator()
    batched_features = list(islice(feature_generator, batch_size))
    while batched_features:
        # Take the batched features and create a batch from it.
        # The batch is a tuple ((inputs), targets), where (inputs)
        # can be (inputs0, inputs1, etc...). each of "inputs*" and
        # "targets" are numpy arrays.
        print('batched_features:' , batched_features)
        print()
        flattened = ([ins[0] for ins in batched_features],
                     [ins[1] for ins in batched_features])
        print('flatenned: ', flattened)
        print()
        flattened_inputs, flattened_targets = flattened
        print(tuple(zip(*flattened_inputs)))
        batch_inputs = tuple(map(np.array, tuple(zip(*flattened_inputs))))
        batch_targets = tuple(map(np.array, tuple(zip(*flattened_targets))))
        yield batch_inputs, batch_targets
        batched_features = list(islice(feature_generator, batch_size))

In [27]:
batches = get_batch_generator(get_feature_generator, batch_size)

In [28]:
j = 0
for i in batches:
    print(i)
    print()
    if(j>10): break
    j = j + 1

batched_features: [([0, 10], [100]), ([1, 11], [101]), ([2, 12], [102]), ([3, 13], [103]), ([4, 14], [104]), ([5, 15], [105]), ([6, 16], [106]), ([7, 17], [107])]

flatenned:  ([[0, 10], [1, 11], [2, 12], [3, 13], [4, 14], [5, 15], [6, 16], [7, 17]], [[100], [101], [102], [103], [104], [105], [106], [107]])

((0, 1, 2, 3, 4, 5, 6, 7), (10, 11, 12, 13, 14, 15, 16, 17))
((array([0, 1, 2, 3, 4, 5, 6, 7]), array([10, 11, 12, 13, 14, 15, 16, 17])), (array([100, 101, 102, 103, 104, 105, 106, 107]),))

batched_features: [([8, 18], [108]), ([9, 19], [109]), ([10, 20], [110]), ([11, 21], [111]), ([12, 22], [112]), ([13, 23], [113]), ([14, 24], [114]), ([15, 25], [115])]

flatenned:  ([[8, 18], [9, 19], [10, 20], [11, 21], [12, 22], [13, 23], [14, 24], [15, 25]], [[108], [109], [110], [111], [112], [113], [114], [115]])

((8, 9, 10, 11, 12, 13, 14, 15), (18, 19, 20, 21, 22, 23, 24, 25))
((array([ 8,  9, 10, 11, 12, 13, 14, 15]), array([18, 19, 20, 21, 22, 23, 24, 25])), (array([108, 109, 110, 11